# Overhead EBUS-Climate Correlations
Previously, I created an area-weighted time series of CO$_{2}$ flux and Adam Phillip's climate indices. This gives us a picture of how the whole EBUS varies with a climate index predictor. However, it prevents us from getting a taste of the spatial correlations. Perhaps a relatively high correlation score is due to a very high correlated cluster. Perhaps there is a gradient in the correlations themselves. Perhaps there are some negative and some positive scores within a given system.

In [1]:
import glob

# Numerics
import numpy as np
import pandas as pd
import xarray as xr
from scipy import stats
from scipy import signal

# Visualization
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.colors import LinearSegmentedColormap
import cmocean

# Functions

In [2]:
def detrend_climate(x):
    return signal.detrend(x)

In [3]:
def smooth_series(x, length=12):
    series = pd.rolling_mean(x, length, center=False)
    series = series[(length-1)::]
    return series

In [4]:
def drop_ensemble_dim(ds, x):
    ds[x] = (('nlat','nlon'), ds[x][0])
    return ds

In [5]:
# Constants
ens = ['001', '002', '009', '010', '011',
       '012', '013', '014', '015', '016',
       '017', '018', '019', '020', '021',
       '022', '023', '024', '025', '026',
       '027', '028', '029', '030', '031',
       '032', '033', '034', '035', '101',
       '102', '103', '104', '105']

# Load in EBUS Regions
Prefiltered to 10 degree Chavez bound and 800km offshore. No smoothing was applied in advance to the files.

In [6]:
EBU = "CalCS"
fileDir = '/glade/p/work/rbrady/EBUS_BGC_Variability/FG_CO2/' + EBU + '/filtered_output/'
fgco2_residuals = xr.open_dataset(fileDir + EBU.lower() + '-FG_CO2-residuals-chavez-800km.nc')

# Load in Other Variable

In [ ]:
VAR = 'SST'
fileDir = '/glade/p/work/rbrady/EBUS_BGC_Variability/' + VAR + '/' + EBU + '/filtered_output/'
var_residuals = xr.open_dataset(fileDir + EBU.lower() + '-' + VAR + '-residuals-chavez-800km.nc')

# Load in Climate Data

In [7]:
fileDir = '/glade/p/work/rbrady/cesmLE_CVDP/extracted_vars/'
ds_cvdp = xr.open_mfdataset(fileDir + '*.nc', decode_times=False, concat_dim='ensemble')

In [8]:
ds_cvdp = ds_cvdp.rename({'npo_pc_mon': 'npo', 
                          'pdo_timeseries_mon': 'pdo',
                          'sam_pc_mon': 'sam',
                          'ipo_timeseries_mon': 'ipo',
                          'amo_timeseries_mon': 'amo',
                          'nao_pc_mon': 'nao'})

In [9]:
times = pd.date_range('1920-01', '2016-01', freq='M')
ds_cvdp['time'] = times
ds_cvdp = ds_cvdp.apply(detrend_climate)

# Linear Regression with Other Variable

In [ ]:
dims = fgco2_residuals['FG_CO2'].shape
results = np.empty([dims[0], dims[2], dims[3]])
results[:] = np.nan

for e in np.arange(0, dims[0], 1): # Ensemble loop
    print "Working on Ensemble " + str(e+1) + " of 34..."
    co2data = fgco2_residuals['FG_CO2'][e]
    vardata = var_residuals[VAR][e]
    for a in np.arange(0, dims[2], 1):
        for b in np.arange(0, dims[3], 1):
            tempco2 = co2data[:, a, b].values
            tempco2 = smooth_series(tempco2)
            tempco2 = tempco2[11::]
            tempvar = vardata[:, a, b].values
            tempvar = smooth_series(tempvar)
            tempvar = tempvar[11::]
            if ~np.isnan(tempco2[0]): # Only perform regression if data are here.
                slope, intercept, r_value, p_value, std_err = stats.linregress(tempvar, tempco2)
                if p_value <= 0.05:
                    results[e,a,b] = r_value
                else:
                    results[e,a,b] = -999 # Insignificant correlation

In [ ]:
np.save('/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/' + 
        EBU.lower() + '/' + EBU.lower() + '-smoothed-fgco2-' + VAR + '-spatial-correlations', results)

# Linear Regression with Climate Indices
Not the most efficient, but gets the job done!

In [ ]:
dims = fgco2_residuals['FG_CO2'].shape
climvar = 'nino34'
results = np.empty([dims[0], dims[2], dims[3]])
results[:] = np.nan

for e in np.arange(0, dims[0], 1): # Ensemble loop
    print "Working on Ensemble " + str(e+1) + " of 34..."
    co2data = fgco2_residuals['FG_CO2'][e]
    climdata = ds_cvdp[climvar][e].values
    climdata = smooth_series(climdata)
    for a in np.arange(0, dims[2], 1):
        for b in np.arange(0, dims[3], 1):
            tempco2 = co2data[:, a, b].values
            tempco2 = smooth_series(tempco2)
            if ~np.isnan(tempco2[0]): # Only perform regression if data are here.
                slope, intercept, r_value, p_value, std_err = stats.linregress(climdata, tempco2)
                if p_value <= 0.05:
                    results[e,a,b] = r_value
                else:
                    results[e,a,b] = -999 # Insignificant correlation

In [ ]:
np.save('/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/' + 
        EBU.lower() + '/spatial_correlations/' + EBU.lower() + '-smoothed-fgco2-smoothed-' + climvar + '-spatial-correlations', results)

# Linear Regression (with Lag)

In [ ]:
LAG = 5

dims = ds_residuals['FG_CO2'].shape
climvar = 'nino34'
results = np.empty([dims[0], dims[2], dims[3]])
results[:] = np.nan

for e in np.arange(0, dims[0], 1): # Ensemble loop
    print "Working on Ensemble " + str(e+1) + " of 34..."
    co2data = ds_residuals['FG_CO2'][e]
    climdata = ds_cvdp[climvar][e][11:] # Accounts for smoothing to CO2 later
    climdata = climdata[:-LAG] # Account for lag.
    for a in np.arange(0, dims[2], 1):
        for b in np.arange(0, dims[3], 1):
            tempco2 = co2data[:, a, b].values
            tempco2 = smooth_series(tempco2)
            tempco2 = tempco2[11:]
            tempco2 = tempco2[LAG::]
            if ~np.isnan(tempco2[0]): # Only perform regression if data are here.
                slope, intercept, r_value, p_value, std_err = stats.linregress(climdata, tempco2)
                if p_value <= 0.05:
                    results[e,a,b] = r_value
                else:
                    results[e,a,b] = -999 # Insignificant correlation

In [ ]:
np.save(EBU.lower() + '-smoothed-fgco2-' + climvar + '-spatial-correlations-lag' + str(LAG), results)

# Visualization Functions

In [10]:
def visual_bounds(EBU):
    if EBU == "CalCS":
        lat1 = 33
        lat2 = 45
        lon1 = -133
        lon2 = -115
    elif EBU == 'HumCS':
        lat1 = -17
        lat2 = -4
        lon1 = -90
        lon2 = -70
    elif EBU == 'CanCS':
        lat1 = 11
        lat2 = 23
        lon1 = -25
        lon2 = -10
    elif EBU == 'BenCS':
        lat1 = -35
        lat2 = -15
        lon1 = 5
        lon2 = 20
    return lat1,lat2,lon1,lon2

In [11]:
def setup_map(ax, lon, lat, lat1=np.nan, lat2=np.nan,
              lon1=np.nan, lon2=np.nan):
    if np.isnan(lat1):
        lat1=np.nanmin(lat)
        lat2=np.nanmax(lat)
        lon1=np.nanmin(lon)
        lon2=np.nanmax(lon)
    m = Basemap(projection='eqdc',
                lat_0=(lat1 + lat2)/2,
                lon_0=(lon1 + lon2)/2,
                llcrnrlon=lon1,
                urcrnrlon=lon2,
                llcrnrlat=lat1,
                urcrnrlat=lat2,
                resolution='c',
                fix_aspect=False,
                ax=ax)
    m.drawcoastlines()
    #m.drawstates()
    m.drawcountries()
    #m.fillcontinents(color='black')
    m.drawmapboundary(fill_color='white')
    #m.drawparallels(np.arange(np.floor(np.nanmin(lat)), 
    #                          np.ceil(np.nanmax(lat)), 3),
    #               labels=[False,False,False,False], size='x-large',
    #               color='gray')
    #m.drawmeridians(np.arange(np.floor(np.nanmin(lon)), 
    #                          np.ceil(np.nanmax(lon)), 3),
    #               labels=[False, False, False, False], size='x-large',
    #               color='gray')
    return m

In [12]:
def pcolor_it(ax, EBU, data):
    fileDir = '/glade/p/work/rbrady/EBUS_BGC_Variability/FG_CO2/' + EBU + '/filtered_residuals/'
    ds_residuals = xr.open_dataset(fileDir + EBU.lower() + '-FG_CO2-residuals-chavez-800km.nc')
    lat = ds_residuals['TLAT'].values
    lon = ds_residuals['TLONG'].values
    data = np.ma.array(data, mask=np.isnan(data))    
    lat1,lat2,lon1,lon2 = visual_bounds(EBU)
    m = setup_map(ax, lon, lat, lat1=lat1, lat2=lat2,
                  lon1=lon1, lon2=lon2)
    return m, lon, lat, data, lat2, lon1

In [13]:
def discrete_cmap(N, base_cmap=None):
    """Create an N-bin discrete colormap from the specified input map"""

    # Note that if base_cmap is a string or None, you can simply do
    #    return plt.cm.get_cmap(base_cmap, N)
    # The following works for string, None, or a colormap instance:

    base = plt.cm.get_cmap(base_cmap)
    color_list = base(np.linspace(0, 1, N))
    cmap_name = base.name + str(N)
    return base.from_list(cmap_name, color_list, N)

# Visualization Code

In [15]:
EBU = 'CalCS'
VAR = 'nino34'
fileDir = '/glade/u/home/rbrady/projects/EBUS_BGC_Variability/data/processed/' + EBU.lower() + '/spatial_correlations/'
results = np.load(fileDir + EBU.lower() + '-smoothed-FG_CO2-smoothed-' + VAR + '-spatial-correlations.npy')

In [ ]:
fig,axes = plt.subplots(figsize=(14,16), nrows=6, ncols=6)
st = fig.suptitle(EBU + ' Grid Cell Correlations with ' + VAR.upper() + ' (1920-2015)', fontsize=30)
counter = 0
for ax in axes.flat:
    if counter >= 34:
        fig.delaxes(ax)
    else:
        m, lon, lat, data, lat2, lon1 = pcolor_it(ax, EBU, results[counter])
        x, y = m(lon1, lat2-1.1)
        ax.text(x, y, 'S' + ens[counter], fontsize=12, 
                bbox=dict(facecolor='w', edgecolor='k', alpha=1))
        cs = m.pcolor(lon, lat, data, vmin=-0.75, vmax=0.75 ,
                     latlon=True, cmap=discrete_cmap(24, 'BrBG_r'), edgecolors='w')
        cs.cmap.set_under('#696969')
    counter+=1
fig.tight_layout(pad=3)
cbar = fig.colorbar(cs, location="top", aspect=50, ax=axes.ravel().tolist())
cbar.ax.tick_params(labelsize=20, direction='out')
cbar.set_ticks(np.arange(-0.75, 0.76, 0.25))
fig.subplots_adjust(top=0.80)
st.set_y(0.95)
plt.savefig('/glade/u/home/rbrady/projects/EBUS_BGC_Variability/reports/figs/' + EBU.lower() + '/' + 'spatial-correlations/'
            + EBU.lower() + '-grid-cell-correlations-smoothed-' + VAR + '-postage.png')